In [ ]:
!pip install emoji

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
test = pd.read_csv("test_emoji.csv", header=None)
train = pd.read_csv("train_emoji.csv" , header = None)

In [ ]:
import emoji

In [ ]:
emoji.EMOJI_UNICODE

In [ ]:
emoji_dictionary = {
    "0" : "\u2764\uFE0F",
    "1" : ":baseball:",
    "2" : ":beaming_face_with_smiling_eyes:",
    "3" : ":downcast_face_with_sweat:",
    "4" : ":fork_and_knife:"
}

In [ ]:
emoji.emojize("\u2764\uFE0F")

In [ ]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

In [ ]:
train.head()

In [ ]:
data = train.values

In [ ]:
for i in range(10):
    print(data[i][0], emoji.emojize(emoji_dictionary[str(data[i][1])]))

In [ ]:
X_train = train[0].values
X_test = test[0].values

In [ ]:
y_train = train[1].values
y_test = test[1].values

In [ ]:
from keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
X_train.shape, y_train.shape

### Get Embeddings

In [ ]:
embeddings = {}

with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.array(values[1:], dtype="float32")
        
        embeddings[word] = coeffs

In [ ]:
embeddings['today']

In [ ]:
def getOutputEmbeddings(X):
    maxlen = 10
    embedding_matrix_output = np.zeros((X.shape[0], maxlen, 50 ))
    
    for i in range(X.shape[0]):
        X[i] = X[i].split()
        
        for j in range(len(X[i])):
            embedding_matrix_output[i][j] = embeddings[X[i][j].lower()]
            
    return embedding_matrix_output

In [ ]:
X_train

In [ ]:
X_train = getOutputEmbeddings(X_train)
X_test = getOutputEmbeddings(X_test)

In [ ]:
X_train.shape

In [ ]:
from keras.layers import *
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape= (10,50), return_sequences= True ))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['acc'])

In [ ]:
hist = model.fit(X_train, y_train, epochs=40, batch_size=32, shuffle=True, validation_split=0.1)

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
test = test[0].values

In [ ]:
for i in range(40):
    print( " ".join(test[i]) , emoji.emojize(emoji_dictionary[str(y_pred[i])]))
    